In [1]:
import time
import requests
import xml.etree.ElementTree as ET

# API 인증키
API_KEY = "Hl9nCtSJsJYpZKSXI1I4S3Ucr7gOPCG6dCX6BSht44RZMRnxVrw4NdAA07lo6kd9fNT6ThCPql8Ia0a2EBDlbw%3D%3D"
# API 공통파라미터
API_COMMON_PARAMS = {"serviceKey" : requests.utils.unquote(API_KEY), "pageNo" : "1"} # service key가 UTF-8형식이므로 decode하여 넘긴다.(get() 내부에서 encoding됨)

CURRENT_DATE = time.strftime("%Y%m%d") # yyyymmdd
BASE_CITY_CODE = "CJU" # 제주

# API 요청
def get_response_xml(url, params):
    response = requests.get(url, params=params)
    print(response.url)
    print(params)
    
    if response.status_code == 200:
        return ET.fromstring(response.text)
    else:
        return None
    
# xml에서 필요한 데이터만 dict형 배열에 저장
def get_data(result_xml, col_names):
    result_list = []
    
    for item in result_xml.findall("./body/items/item"):
        result = {}
        for xml_col_name, table_col_name in col_names.items():
            result[table_col_name] = item.find(xml_col_name).text
            
        result_list.append(result)
            
    return result_list
    
# 국내선 운항 스케쥴(제주 출발) 조회(/FlightScheduleList/getDflightScheduleList)
def get_departure_schedule_list():
    # 가져올 데이터의 항목명 정의
    col_name = {"airlineKorean" : "airline_name",
                "arrivalcity" : "arrival_city",
                "domesticNum" : "flight_num",
                "domesticArrivalTime" : "arrival_time",
                "domesticStartTime" : "departure_time",
                "domesticStdate" : "start_date",
                "domesticEddate" : "end_date",
                "domesticFri" : "friday_yn"
               }
    params = API_COMMON_PARAMS.copy()
    params["schDate"] = CURRENT_DATE
    params["schDeptCityCode"] = BASE_CITY_CODE
    params["numOfRows"] = 999 # 한번에 출력할 최대 데이터수(*이 값이 상은 안넘어갈거 같아서 우선 대충 설정..)
    url = "http://openapi.airport.co.kr/service/rest/FlightScheduleList/getDflightScheduleList"
    
    result_xml = get_response_xml(url, params)
    
    return get_data(result_xml, col_name)

# 국내선 운항 스케쥴(제주 도착) 조회(/FlightScheduleList/getDflightScheduleList)
def get_arrival_schedule_list():
    col_name = {"airlineKorean" : "airline_name",
                "startcity" : "departure_city",
                "domesticNum" : "flight_num",
                "domesticArrivalTime" : "arrival_time",
                "domesticStartTime" : "departure_time",
                "domesticStdate" : "start_date",
                "domesticEddate" : "end_date",
                "domesticFri" : "friday_yn"
               }
    params = API_COMMON_PARAMS.copy()
    params["schDate"] = CURRENT_DATE
    params["schArrvCityCode"] = BASE_CITY_CODE
    params["numOfRows"] = 999 # 한번에 출력할 최대 데이터수(*이 값이 상은 안넘어갈거 같아서 우선 대충 설정..)
    url = "http://openapi.airport.co.kr/service/rest/FlightScheduleList/getDflightScheduleList"
    
    result_xml = get_response_xml(url, params)
    
    return get_data(result_xml, col_name)

# 실시간 운항정보 조회(/FlightStatusList/getFlightStatusList)
def get_flight_status_list():
    # 가져올 데이터의 항목명
    col_name = {"airFln" : "flight_num",
                "airlineKorean" : "airline_name",
                "airport" : "airport_code",
                "city" : "city_code",
                "arrivedKor" : "arrived_airport_name",
                "boardingKor" : "boarding_airport_name",
                "std" : "scheduled_time",
                "etd" : "estimated_time",
                "io" : "inout_code",
                "rmkEng" : "flight_status_eng",
                "rmkKor" : "flight_status_kor"                
               }
    params = API_COMMON_PARAMS.copy()
    params["schLineType"] = "D" # D:국내
    params["schAirCode"] = BASE_CITY_CODE
    params["numOfRows"] = 999 # 한번에 출력할 최대 데이터수(*이 값이 상은 안넘어갈거 같아서 우선 대충 설정..)
    url = "http://openapi.airport.co.kr/service/rest/FlightStatusList/getFlightStatusList"
    
    result_xml = get_response_xml(url, params)
    
    return get_data(result_xml, col_name)


In [2]:
departure_list = get_departure_schedule_list()
arrival_list = get_arrival_schedule_list()
status_list = get_flight_status_list()

http://openapi.airport.co.kr/service/rest/FlightScheduleList/getDflightScheduleList?serviceKey=Hl9nCtSJsJYpZKSXI1I4S3Ucr7gOPCG6dCX6BSht44RZMRnxVrw4NdAA07lo6kd9fNT6ThCPql8Ia0a2EBDlbw%3D%3D&pageNo=1&schDate=20200409&schDeptCityCode=CJU&numOfRows=999
{'serviceKey': 'Hl9nCtSJsJYpZKSXI1I4S3Ucr7gOPCG6dCX6BSht44RZMRnxVrw4NdAA07lo6kd9fNT6ThCPql8Ia0a2EBDlbw==', 'pageNo': '1', 'schDate': '20200409', 'schDeptCityCode': 'CJU', 'numOfRows': 999}
http://openapi.airport.co.kr/service/rest/FlightScheduleList/getDflightScheduleList?serviceKey=Hl9nCtSJsJYpZKSXI1I4S3Ucr7gOPCG6dCX6BSht44RZMRnxVrw4NdAA07lo6kd9fNT6ThCPql8Ia0a2EBDlbw%3D%3D&pageNo=1&schDate=20200409&schArrvCityCode=CJU&numOfRows=999
{'serviceKey': 'Hl9nCtSJsJYpZKSXI1I4S3Ucr7gOPCG6dCX6BSht44RZMRnxVrw4NdAA07lo6kd9fNT6ThCPql8Ia0a2EBDlbw==', 'pageNo': '1', 'schDate': '20200409', 'schArrvCityCode': 'CJU', 'numOfRows': 999}
http://openapi.airport.co.kr/service/rest/FlightStatusList/getFlightStatusList?serviceKey=Hl9nCtSJsJYpZKSXI1I4S3Ucr7gOPCG6dCX6

In [3]:
departure_list

[{'airline_name': '아시아나항공',
  'arrival_city': '서울/김포',
  'flight_num': 'OZ8900',
  'arrival_time': '0740',
  'departure_time': '0630',
  'start_date': '2020-03-29T00:00:00+09:00',
  'end_date': '2020-06-22T00:00:00+09:00',
  'friday_yn': 'Y'},
 {'airline_name': '아시아나항공',
  'arrival_city': '서울/김포',
  'flight_num': 'OZ8902',
  'arrival_time': '0750',
  'departure_time': '0640',
  'start_date': '2020-03-29T00:00:00+09:00',
  'end_date': '2020-10-24T00:00:00+09:00',
  'friday_yn': 'Y'},
 {'airline_name': '진에어',
  'arrival_city': '광주',
  'flight_num': 'LJ592',
  'arrival_time': '0810',
  'departure_time': '0720',
  'start_date': '2020-03-29T00:00:00+09:00',
  'end_date': '2020-04-25T00:00:00+09:00',
  'friday_yn': 'Y'},
 {'airline_name': '에어서울',
  'arrival_city': '서울/김포',
  'flight_num': 'RS902',
  'arrival_time': '0840',
  'departure_time': '0730',
  'start_date': '2020-03-29T00:00:00+09:00',
  'end_date': '2020-10-24T00:00:00+09:00',
  'friday_yn': 'N'},
 {'airline_name': '에어서울',
  'arriv

In [4]:
print(departure_list[0])
print(arrival_list[0])
print(status_list[0])
print(len(departure_list))
print(len(arrival_list))
print(len(status_list))

{'airline_name': '아시아나항공', 'arrival_city': '서울/김포', 'flight_num': 'OZ8900', 'arrival_time': '0740', 'departure_time': '0630', 'start_date': '2020-03-29T00:00:00+09:00', 'end_date': '2020-06-22T00:00:00+09:00', 'friday_yn': 'Y'}
{'airline_name': '에어서울', 'departure_city': '서울/김포', 'flight_num': 'RS901', 'arrival_time': '0700', 'departure_time': '0600', 'start_date': '2020-03-29T00:00:00+09:00', 'end_date': '2020-10-24T00:00:00+09:00', 'friday_yn': 'N'}
{'flight_num': 'OZ8900', 'airline_name': '아시아나항공', 'airport_code': 'CJU', 'city_code': 'GMP', 'arrived_airport_name': '서울/김포', 'boarding_airport_name': '제주', 'scheduled_time': '0630', 'estimated_time': '0640', 'inout_code': 'O', 'flight_status_eng': 'DEPARTED', 'flight_status_kor': '출발'}
192
217
196
